In [1]:
import json
import pandas as pd
import numpy as np
import torch

In [2]:
def get_file(filename):
    data = {}
    try:
        with open(filename, 'r') as f:
            data = json.load(f)
        print("JSON data loaded successfully:")
    except Exception as e:
        print(e)
    return {"data": data}

In [3]:
node_new_Twibot22 = get_file("Twibot-22/node_new.json")
label_new_Twibot22 = get_file("Twibot-22/label_new.json")

JSON data loaded successfully:
JSON data loaded successfully:


In [12]:
point1 = node_new_Twibot22["data"]["t1498018021658431488"]
point1_author_id = point1["author_id"]

In [20]:
# the tweet
print(point1)
# is the author at key "u" + author_id a bot or not? 
label_new_Twibot22["data"]["u"+str(point1_author_id)]

{'attachments': None, 'author_id': 20441260, 'context_annotations': None, 'conversation_id': 1498018021658431488, 'created_at': '2022-02-27 19:31:42+00:00', 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'rhpsia', 'name': 'Sia At The Game', 'id': 809332867, 'id_str': '809332867', 'indices': [3, 10]}], 'urls': []}, 'geo': None, 'id': 't1498018021658431488', 'in_reply_to_user_id': None, 'lang': 'en', 'possibly_sensitive': False, 'public_metrics': {'retweet_count': 10, 'reply_count': None, 'like_count': 0, 'quote_count': None}, 'referenced_tweets': None, 'reply_settings': None, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'text': 'RT @rhpsia: BREAKING: Telecom giant Ericsson used slush funds, trips for Iraqi officials &amp; payoffs through middlemen to executives — and po…', 'withheld': None}


'human'

In [21]:
# Count top-level keys in Twibot-22/node_new.json that start with 't'
try:
    # node_new_Twibot22 is expected to be a dict returned by get_file() earlier in this notebook
    data_obj = node_new_Twibot22 if not isinstance(node_new_Twibot22, dict) else node_new_Twibot22.get("data", node_new_Twibot22)
except NameError:
    # If the variable isn't defined (cell not run), load the file now
    print("node_new_Twibot22 not found in the notebook state - loading from file")
    node_new_Twibot22 = get_file("Twibot-22/node_new.json")
    data_obj = node_new_Twibot22.get("data", node_new_Twibot22)

# Ensure we have a mapping-like object
if not isinstance(data_obj, dict):
    print("Unexpected format: expected a JSON object/dict for node data.")
else:
    keys = [k for k in data_obj.keys() if isinstance(k, str) and k.startswith("t")]
    count = len(keys)
    print(f"Found {count} keys starting with 't' in Twibot-22/node_new.json")
    print(f"Scanned {len(data_obj)} top-level keys in the node object.")
    # Show up to 10 sample keys to verify
    print("Sample keys (up to 10):", keys[:10])

Found 31228 keys starting with 't' in Twibot-22/node_new.json
Scanned 34936 top-level keys in the node object.
Sample keys (up to 10): ['t1498018021658431488', 't1497947823005962245', 't1497902014763786241', 't1497736000008867841', 't1497640323631222784', 't1497417782450864130', 't1497402632301854722', 't1497288192969654273', 't1497277930623361033', 't1497275497126019076']


In [ ]:
# add a label to every tweet key
num_bot = 0 
num_human = 0
num_unlabeled = 0

for cur_tweet_id in keys: 
    cur_tweet = node_new_Twibot22["data"][cur_tweet_id]
    cur_author_id = cur_tweet["author_id"]
    
    if "u"+str(cur_author_id) in label_new_Twibot22["data"]: 
        cur_author_label = label_new_Twibot22["data"]["u"+str(cur_author_id)]
    else:
        num_error+= 1 # ids nonexistent in labels

    if (cur_author_label == "human"):
        num_human += 1
    if (cur_author_label == "bot"):
        num_bot += 1

    node_new_Twibot22["data"][cur_tweet_id]["author_label"] = cur_author_label

print(num_bot)
print(num_human)
print(num_unlabeled)

8171
23057
644


There are 8171 bot tweets, and 23057 human tweets


In [44]:
# from pyspark.ml.feature import Tokenizer, CountVectorizer
# from pyspark.ml.classification import LogisticRegression

# tokenizer = Tokenizer(inputCol="text", outputCol="words")
# cv = CountVectorizer(inputCol="words", outputCol="features")
# lr = LogisticRegression(featuresCol="features", labelCol="label")